In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
print("=" * 80)
print("DATA PREPROCESSING ANALYSIS REPORT")
print("=" * 80)
print("\n1. LOADING DATASET...")
df = pd.read_csv('filter-jobs-requiring-studies.csv')
print(f"   Original dataset shape: {df.shape}")
print(f"   Columns: {list(df.columns)}")
print(f"   Total records: {len(df):,}")


DATA PREPROCESSING ANALYSIS REPORT

1. LOADING DATASET...
   Original dataset shape: (542650, 7)
   Columns: ['person_id', 'matched_label', 'matched_description', 'matched_code', 'start_date', 'end_date', 'university_studies']
   Total records: 542,650


In [2]:
# Initial Data Analysis
print("\n2. INITIAL DATA ANALYSIS")
print("-" * 80)

# Check data types
print("\n2.1 Data Types:")
print(df.dtypes)

# Check for null values
print("\n2.2 Null Values Count:")
null_counts = df.isnull().sum()
print(null_counts[null_counts > 0] if null_counts.sum() > 0 else "No null values found")

# Check for empty strings
print("\n2.3 Empty Strings Count:")
empty_strings = {}
for col in df.columns:
    if df[col].dtype == 'object':
        empty_count = (df[col] == '').sum()
        if empty_count > 0:
            empty_strings[col] = empty_count
print(empty_strings if empty_strings else "No empty strings found")

# Check for 'unknown' values (case-insensitive)
print("\n2.4 'Unknown' Values Count (case-insensitive):")
unknown_counts = {}
for col in df.columns:
    if df[col].dtype == 'object':
        unknown_count = df[col].astype(str).str.lower().str.strip().isin(['unknown', 'nan', 'none']).sum()
        if unknown_count > 0:
            unknown_counts[col] = unknown_count
print(unknown_counts if unknown_counts else "No 'unknown' values found")

# Display sample of problematic rows
print("\n2.5 Sample Rows with Issues:")
issues_mask = (
    df.isnull().any(axis=1) | 
    (df.select_dtypes(include=['object']).astype(str).apply(lambda x: x.str.lower().str.strip().isin(['unknown', 'nan', 'none'])).any(axis=1)) |
    (df.select_dtypes(include=['object']).astype(str).apply(lambda x: x == '').any(axis=1))
)
if issues_mask.sum() > 0:
    print(f"   Found {issues_mask.sum():,} rows with issues")
    print(df[issues_mask].head(10))
else:
    print("   No issues found in sample")



2. INITIAL DATA ANALYSIS
--------------------------------------------------------------------------------

2.1 Data Types:
person_id               int64
matched_label          object
matched_description    object
matched_code           object
start_date             object
end_date               object
university_studies       bool
dtype: object

2.2 Null Values Count:
matched_code      5097
start_date       52740
end_date        121141
dtype: int64

2.3 Empty Strings Count:
No empty strings found

2.4 'Unknown' Values Count (case-insensitive):
{'matched_label': 87313, 'matched_description': 87313, 'matched_code': 87313, 'start_date': 52740, 'end_date': 121141}

2.5 Sample Rows with Issues:
   Found 206,203 rows with issues
    person_id matched_label  \
13          1       unknown   
16          2    accountant   
24          9       unknown   
33         14       unknown   
34         14       unknown   
35         14       unknown   
36         14       unknown   
37         14     

In [3]:
# Check for inconsistencies in date columns
print("\n3. CHECKING FOR INCONSISTENCIES")
print("-" * 80)

# Date format validation
print("\n3.1 Date Format Analysis:")
date_columns = ['start_date', 'end_date']

for col in date_columns:
    if col in df.columns:
        # Check for valid date format (Q1 YYYY, Q2 YYYY, etc.)
        valid_format = df[col].astype(str).str.match(r'^Q[1-4]\s+\d{4}$', na=False)
        invalid_count = (~valid_format & df[col].notna()).sum()
        print(f"   {col}: {invalid_count:,} invalid format entries")
        if invalid_count > 0:
            invalid_samples = df[~valid_format & df[col].notna()][col].unique()[:5]
            print(f"      Sample invalid formats: {list(invalid_samples)}")

# Check for logical inconsistencies (end_date before start_date)
print("\n3.2 Date Logic Validation:")
if 'start_date' in df.columns and 'end_date' in df.columns:
    # Convert dates to comparable format
    def parse_quarter(date_str):
        if pd.isna(date_str) or str(date_str).strip() == '':
            return None
        try:
            parts = str(date_str).strip().split()
            if len(parts) == 2 and parts[0].startswith('Q'):
                quarter = int(parts[0][1])
                year = int(parts[1])
                return year * 4 + quarter
            return None
        except:
            return None
    
    df['start_quarter_num'] = df['start_date'].apply(parse_quarter)
    df['end_quarter_num'] = df['end_date'].apply(parse_quarter)
    
    # Check for invalid date ranges
    invalid_ranges = (
        (df['start_quarter_num'].notna() & df['end_quarter_num'].notna()) &
        (df['end_quarter_num'] < df['start_quarter_num'])
    )
    print(f"   Found {invalid_ranges.sum():,} rows where end_date is before start_date")
    if invalid_ranges.sum() > 0:
        print(f"   Sample invalid date ranges:")
        print(df[invalid_ranges][['person_id', 'start_date', 'end_date']].head(5))

# Check for duplicate rows
print("\n3.3 Duplicate Rows:")
duplicate_count = df.duplicated().sum()
print(f"   Found {duplicate_count:,} duplicate rows")

# Check for inconsistent data types in matched_code
print("\n3.4 Matched Code Validation:")
if 'matched_code' in df.columns:
    # Check if codes follow expected pattern (numbers with dots)
    code_pattern = df['matched_code'].astype(str).str.match(r'^\d+\.?\d*\.?\d*\.?\d*$', na=False)
    invalid_codes = (~code_pattern & df['matched_code'].notna() & (df['matched_code'].astype(str).str.strip() != '')).sum()
    print(f"   Found {invalid_codes:,} entries with unexpected code format")
    if invalid_codes > 0:
        invalid_samples = df[~code_pattern & df['matched_code'].notna() & (df['matched_code'].astype(str).str.strip() != '')]['matched_code'].unique()[:5]
        print(f"   Sample invalid codes: {list(invalid_samples)}")



3. CHECKING FOR INCONSISTENCIES
--------------------------------------------------------------------------------

3.1 Date Format Analysis:
   start_date: 0 invalid format entries
   end_date: 0 invalid format entries

3.2 Date Logic Validation:
   Found 12 rows where end_date is before start_date
   Sample invalid date ranges:
        person_id start_date end_date
14122       11599    Q1 2011  Q4 2010
93960       77134    Q1 2016  Q4 2015
127009     103824    Q1 2008  Q4 2007
260040     213714    Q1 2014  Q4 2012
276401     227068    Q1 2007  Q4 1993

3.3 Duplicate Rows:
   Found 13,528 duplicate rows

3.4 Matched Code Validation:
   Found 83,242 entries with unexpected code format
   Sample invalid codes: ['unknown', '1324.3.1.6.10', '1221.3.2.1.1', '1324.3.1.6.7', '1221.3.2.1.4']


In [4]:
# CLEANING THE DATA
print("\n4. CLEANING THE DATA")
print("-" * 80)

# Create a copy for cleaning
df_clean = df.copy()
initial_rows = len(df_clean)

# Step 1: Remove rows with 'unknown' values (case-insensitive)
print("\n4.1 Removing rows with 'unknown' values...")
before_unknown = len(df_clean)
unknown_mask = (
    df_clean['matched_label'].astype(str).str.lower().str.strip().isin(['unknown', 'nan', 'none']) |
    df_clean['matched_description'].astype(str).str.lower().str.strip().isin(['unknown', 'nan', 'none']) |
    df_clean['matched_code'].astype(str).str.lower().str.strip().isin(['unknown', 'nan', 'none'])
)
df_clean = df_clean[~unknown_mask]
removed_unknown = before_unknown - len(df_clean)
print(f"   Removed {removed_unknown:,} rows with 'unknown' values")

# Step 2: Remove null values
print("\n4.2 Removing rows with null values...")
before_null = len(df_clean)
df_clean = df_clean.dropna()
removed_null = before_null - len(df_clean)
print(f"   Removed {removed_null:,} rows with null values")

# Step 3: Remove empty strings
print("\n4.3 Removing rows with empty strings...")
before_empty = len(df_clean)
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        df_clean = df_clean[df_clean[col].astype(str).str.strip() != '']
removed_empty = before_empty - len(df_clean)
print(f"   Removed {removed_empty:,} rows with empty strings")

# Step 4: Remove rows with invalid date formats
print("\n4.4 Removing rows with invalid date formats...")
before_date = len(df_clean)
date_pattern = r'^Q[1-4]\s+\d{4}$'
valid_start = df_clean['start_date'].astype(str).str.match(date_pattern, na=False)
valid_end = df_clean['end_date'].astype(str).str.match(date_pattern, na=False)
df_clean = df_clean[valid_start & valid_end]
removed_date = before_date - len(df_clean)
print(f"   Removed {removed_date:,} rows with invalid date formats")

# Step 5: Remove rows with invalid date ranges (end_date before start_date)
print("\n4.5 Removing rows with invalid date ranges...")
before_range = len(df_clean)

# Define helper function to parse quarter dates
def parse_quarter(date_str):
    if pd.isna(date_str) or str(date_str).strip() == '':
        return None
    try:
        parts = str(date_str).strip().split()
        if len(parts) == 2 and parts[0].startswith('Q'):
            quarter = int(parts[0][1])
            year = int(parts[1])
            return year * 4 + quarter
        return None
    except:
        return None

df_clean['start_quarter_num'] = df_clean['start_date'].apply(parse_quarter)
df_clean['end_quarter_num'] = df_clean['end_date'].apply(parse_quarter)
invalid_ranges = (
    (df_clean['start_quarter_num'].notna() & df_clean['end_quarter_num'].notna()) &
    (df_clean['end_quarter_num'] < df_clean['start_quarter_num'])
)
df_clean = df_clean[~invalid_ranges]
removed_range = before_range - len(df_clean)
print(f"   Removed {removed_range:,} rows with invalid date ranges")

# Step 6: Remove duplicate rows
print("\n4.6 Removing duplicate rows...")
before_dup = len(df_clean)
df_clean = df_clean.drop_duplicates()
removed_dup = before_dup - len(df_clean)
print(f"   Removed {removed_dup:,} duplicate rows")

# Clean up temporary columns
df_clean = df_clean.drop(columns=['start_quarter_num', 'end_quarter_num'], errors='ignore')

final_rows = len(df_clean)
total_removed = initial_rows - final_rows

print(f"\n   Total rows removed: {total_removed:,}")
print(f"   Final dataset shape: {df_clean.shape}")
print(f"   Data retention: {(final_rows/initial_rows*100):.2f}%")



4. CLEANING THE DATA
--------------------------------------------------------------------------------

4.1 Removing rows with 'unknown' values...
   Removed 87,313 rows with 'unknown' values

4.2 Removing rows with null values...
   Removed 118,890 rows with null values

4.3 Removing rows with empty strings...
   Removed 0 rows with empty strings

4.4 Removing rows with invalid date formats...
   Removed 0 rows with invalid date formats

4.5 Removing rows with invalid date ranges...
   Removed 5 rows with invalid date ranges

4.6 Removing duplicate rows...
   Removed 3,058 duplicate rows

   Total rows removed: 209,266
   Final dataset shape: (333384, 7)
   Data retention: 61.44%


In [5]:
# Final Data Quality Report
print("\n5. FINAL DATA QUALITY REPORT")
print("=" * 80)

print(f"\nDataset Statistics:")
print(f"   Original rows: {initial_rows:,}")
print(f"   Cleaned rows: {final_rows:,}")
print(f"   Rows removed: {total_removed:,} ({(total_removed/initial_rows*100):.2f}%)")

print(f"\nColumn-wise Summary:")
for col in df_clean.columns:
    null_count = df_clean[col].isnull().sum()
    if df_clean[col].dtype == 'object':
        empty_count = (df_clean[col].astype(str).str.strip() == '').sum()
        unknown_count = df_clean[col].astype(str).str.lower().str.strip().isin(['unknown', 'nan', 'none']).sum()
        print(f"   {col}:")
        print(f"      - Null values: {null_count}")
        print(f"      - Empty strings: {empty_count}")
        print(f"      - Unknown values: {unknown_count}")
        print(f"      - Unique values: {df_clean[col].nunique():,}")
    else:
        print(f"   {col}:")
        print(f"      - Null values: {null_count}")
        print(f"      - Data type: {df_clean[col].dtype}")

print(f"\nData Quality Checks:")
print(f"   ✓ No null values: {df_clean.isnull().sum().sum() == 0}")
print(f"   ✓ No empty strings: {not (df_clean.select_dtypes(include=['object']).astype(str).apply(lambda x: x.str.strip() == '').any().any())}")
print(f"   ✓ No 'unknown' values: {not (df_clean.select_dtypes(include=['object']).astype(str).apply(lambda x: x.str.lower().str.strip().isin(['unknown', 'nan', 'none'])).any().any())}")
print(f"   ✓ No duplicates: {not df_clean.duplicated().any()}")
print(f"   ✓ Valid date formats: All dates follow Q[1-4] YYYY format")
print(f"   ✓ Valid date ranges: All end_dates are after start_dates")

print("\n" + "=" * 80)
print("PREPROCESSING COMPLETE!")
print("=" * 80)



5. FINAL DATA QUALITY REPORT

Dataset Statistics:
   Original rows: 542,650
   Cleaned rows: 333,384
   Rows removed: 209,266 (38.56%)

Column-wise Summary:
   person_id:
      - Null values: 0
      - Data type: int64
   matched_label:
      - Null values: 0
      - Empty strings: 0
      - Unknown values: 0
      - Unique values: 2,799
   matched_description:
      - Null values: 0
      - Empty strings: 0
      - Unknown values: 0
      - Unique values: 2,799
   matched_code:
      - Null values: 0
      - Empty strings: 0
      - Unknown values: 0
      - Unique values: 2,799
   start_date:
      - Null values: 0
      - Empty strings: 0
      - Unknown values: 0
      - Unique values: 205
   end_date:
      - Null values: 0
      - Empty strings: 0
      - Unknown values: 0
      - Unique values: 196
   university_studies:
      - Null values: 0
      - Data type: bool

Data Quality Checks:
   ✓ No null values: True
   ✓ No empty strings: True
   ✓ No 'unknown' values: True
   ✓ 

In [6]:
# Save cleaned dataset
print("\n6. SAVING CLEANED DATASET")
print("-" * 80)

output_filename = 'filter-jobs-requiring-studies-cleaned.csv'
df_clean.to_csv(output_filename, index=False)
print(f"   Cleaned dataset saved to: {output_filename}")
print(f"   File size: {len(df_clean):,} rows × {len(df_clean.columns)} columns")

# Display sample of cleaned data
print("\n7. SAMPLE OF CLEANED DATA")
print("-" * 80)
print(df_clean.head(10))



6. SAVING CLEANED DATASET
--------------------------------------------------------------------------------
   Cleaned dataset saved to: filter-jobs-requiring-studies-cleaned.csv
   File size: 333,384 rows × 7 columns

7. SAMPLE OF CLEANED DATA
--------------------------------------------------------------------------------
   person_id                              matched_label  \
0          0                           resource manager   
1          0                  health and safety officer   
2          0                       integration engineer   
3          0                          programme manager   
4          0    product development engineering drafter   
5          0                               move manager   
6          0  customer contact centre information clerk   
7          0                      ICT help desk manager   
8          0                        ICT help desk agent   
9          0                    language school teacher   

                        